# Playful: Find your new favorite computer game
I built [an app that recommends computer games on Steam](http://www.playful.live/).

To do that, I obtained Steam data and performed some initial data exploration using
* scrapy
* API calls
* PostgreSQL

Next I generated, optimized, and validated a model using
* matrix factorization
* collaborative filtering with implicit feedback
* sparse matrices
* LightFM

Finally I built a web app to turn that model into recommendations for anyone who owns games on Steam using:
* item-to-item recommendations 
* pandas
* flask
* Amazon web services

Here is a more detailed overview of what that process looked like.

## Import stuff
My config.py file is not on GitHub. You need your own Steam API key and database information.

In [1]:
import json
import pandas as pd
from app.config import api_key, db_username, db_password, db_host, db_port
from urllib.request import Request, urlopen
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

## Scrape reviews for user IDs
Scraping hub has [a detailed example of how to scrape reviews from the Steam store using scrapy]((https://blog.scrapinghub.com/2017/07/07/scraping-the-steam-game-store-with-scrapy/), complete with code in a GitHub repo. 

I scraped all of the reviews, which took about 4 days, in case later on I want to incorporate some of that information into the recommendations. For now the only thing I'm using from that exercize is a list of ~400,000 unique Steam user IDs of the review writers. I did not include any other Steam users, so my recommendations are biased toward games owned by people who have written reviews. 

Due to space limitations on GitHub, I am sharing only a small part of 1 of the 3 scrapy output files.

In [2]:
def load_reviews():
    reviews = []
    path_to_scraped_data = 'example_data//'
    files = ['scraped_reviews.jl']
    
    for file in files:
        with open(''.join((path_to_scraped_data, file)), 'r') as f:
            for line in f:
                reviews.append(json.loads(line))
                
    return reviews

scraped_reviews = load_reviews()

user_ids = []
for review in scraped_reviews:
    try:
        user_ids.append(review['user_id'])
    except KeyError:
        pass
    
unique_users = list(set(user_ids))
print('There are', len(unique_users), 'unique steam user IDs in the sample data.')

There are 1094 unique steam user IDs in the sample data.


## API calls for game ownership
This took about 5 minutes, and you have to be online.

In [3]:
def getGamesOwned(player_id):
    req = Request('http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=%s&steamid=%s&format=json&include_played_free_games=True&include_appinfo=True'%(api_key, player_id))
    try:
        data_raw = urlopen(req).read()
        data_json = json.loads(data_raw)
        return data_json['response']['games']
    except:
        return []

def get_all_games_owned_by_players(user_ids):
    users_and_their_games = {}
    for idx, gamer_id in enumerate(user_ids):
        users_and_their_games[gamer_id] = getGamesOwned(gamer_id)
    return users_and_their_games

users_and_their_games = get_all_games_owned_by_players(unique_users)

## Put the ownership data into pandas and PostgreSQL
Every user-game pair gets its own row in the database. For example, say I have data for only 2 unique Steam users, Katie and Minchun. If Katie owns 20 games and Minchun owns 3 games, I'll end up with 23 rows. 

You have to have a SQL server installed and running with apppropriate password information for this section to work. Also, I used Windows. The syntax will be different on a Mac or Linux.

In [ ]:
user_column = []
app_column = []

for user in unique_users:
    for game in users_and_their_games[user]:
        user_column.append(user)
        app_column.append(game['appid'])

user_game_df = pd.DataFrame({'user_id':user_column, 'app_id':app_column})

db_name  = 'playful'
engine = create_engine('postgresql+psycopg2://%s:%s@%s:%s/%s'%(db_username,db_password,db_host,db_port,db_name))

if not database_exists(engine.url):
    create_database(engine.url)
    
user_game_df.to_sql('user_games_table', engine, if_exists='replace')

user_game_df.head()

## SQL query for most popular games
This is how I came up with the list of the 12 most popular games on the app homepage.

At scale, this SQL query was much faster than a similar analysis in pandas.

In [ ]:
sql_query = """ SELECT app_id, COUNT(user_id) AS "n_owners"
                FROM user_games_table
                GROUP BY app_id
                ORDER BY n_owners DESC
                LIMIT 12
            """

con = None
con = psycopg2.connect(database=db_name, user=db_username, password=db_password, host=db_host, port=db_port)
most_popular_game_ids = pd.read_sql_query(sql_query, con).app_id.values

## More details coming soon!